In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('charity_navigator_clean.csv', index_col = 0)

In [3]:
n = 0
for excess_or_deficit in df['excess_or_deficit_for_year']:
    if df.loc[n, 'excess_or_deficit_for_year'] > 0:
        df.loc[n, 'excess_or_deficit_id'] = str(1)
        n += 1
    elif df.loc[n, 'excess_or_deficit_for_year'] <= 0:
        df.loc[n, 'excess_or_deficit_id'] = str(0)
        n += 1

In [4]:
df2 = df[df['compensation_leader_compensation'].isnull() == False]
org_type_dummies = pd.get_dummies(df2.org_type)
df2 = pd.concat([df2, org_type_dummies], axis=1, join = 'outer')
df2.drop(['charity_name', 'charity_url', 
          'city', 'cn_advisory', 
          'state', 'org_type',
          'org_category', 'compensation_leader_title'], 
         axis = 1, inplace = True)
df2.head()

accountability_score  administrative_expenses  \
0                  89.0                 164590.0   
1                  86.0                1001560.0   
2                  85.0                  93957.0   
3                  86.0                 346867.0   
4                  97.0                 135195.0   

   compensation_leader_compensation  compensation_leader_expense_percent  \
0                           53463.0                                 3.11   
1                           73500.0                                 1.47   
2                           85000.0                                 0.99   
3                           61220.0                                 0.78   
4                           74244.0                                 5.41   

   excess_or_deficit_for_year  financial_score  fundraising_expenses  \
0                    349718.0            90.56                111522   
1                   1175965.0            85.92                 54613   
2                   -461502.0            77.65                248833   
3                   1872733.0            97.50                384550   
4                   -103940.0            87.08                 87436   

   net_assets  other_revenue  overall_score             ...              \
0     1350382        49634.0          89.75             ...               
1    14773920       382540.0          85.95             ...               
2     -770370            0.0          80.96             ...               
3    11460087       -81726.0          89.94             ...               
4      723772        32436.0          90.62             ...               

   Arts, Culture, Humanities  Community Development  Education  Environment  \
0                          0                      0          0            0   
1                          0                      0          1            0   
2                          0                      0          0            0   
3                          0                      0          1            0   
4                          0                      0          0            0   

  Health  Human Services  Human and Civil Rights  International  Religion  \
0      0               0                       1              0         0   
1      0               0                       0              0         0   
2      0               0                       0              1         0   
3      0               0                       0              0         0   
4      0               0                       0              0         1   

   Research and Public Policy  
0                           0  
1                           0  
2                           0  
3                           0  
4                           0  

[5 rows x 26 columns]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [6]:
X = df2[['accountability_score', 'financial_score',
         'overall_score', 'administrative_expenses',
         'fundraising_expenses', 'net_assets',
         'other_revenue', 'payments_to_affiliates',
         'compensation_leader_compensation', 
         'compensation_leader_expense_percent',
         'Animals', 'Arts, Culture, Humanities',
         'Community Development',
         'Education', 'Environment',
         'Health', 'Human Services',
         'Human and Civil Rights', 'International',
         'Religion', 'Research and Public Policy']]
y = df2['excess_or_deficit_id']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 101)
logmodel = LogisticRegression().fit(X_train, y_train)
y_pred = logmodel.predict(X_test)

print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logmodel.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logmodel.score(X_test, y_test)))

Accuracy of Logistic regression classifier on training set: 0.57
Accuracy of Logistic regression classifier on test set: 0.57


In [8]:
print('Train R^2:', logmodel.score(X_train, y_train).round(4))
print('Test  R^2:', logmodel.score(X_test, y_test).round(4))
print('MAE:      ', metrics.mean_absolute_error(y_test, y_pred).round(4))
print('MSE:      ', metrics.mean_squared_error(y_test, y_pred).round(4))
print('RMSE:     ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(4))

Train R^2: 0.5706
Test  R^2: 0.5706
MAE:       0.4294
MSE:       0.4294
RMSE:      0.6553


In [9]:
print(metrics.classification_report(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.23      0.00      0.00      1245
          1       0.57      0.99      0.73      1671

avg / total       0.43      0.57      0.42      2916

[[   3 1242]
 [  10 1661]]
